In [7]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import glob
import cv2
from keras.models import load_model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import DenseNet121

In [11]:
dataset_path = '2023sumdpl302m/devset_images/devset_images/'

image_width = 224
image_height = 224

# Load the CSV file with image IDs and labels
csv_file = pd.read_csv('2023sumdpl302m/devset_images_gt.csv')
image_ids = csv_file['id'].values
labels = csv_file['label'].values

In [13]:
images = []
y = []

# Load and preprocess images
for image_id, label in zip(image_ids, labels):
    # Search for files with multiple extensions
    image_path = os.path.join(dataset_path, f'{image_id}.jpg')
    if not os.path.isfile(image_path):
        image_path = os.path.join(dataset_path, f'{image_id}.png')
        if not os.path.isfile(image_path):
            image_path = os.path.join(dataset_path, f'{image_id}.gif')

    if os.path.isfile(image_path):
        # Load the image using OpenCV
        try:
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        except:
            # Retry with PNG extension
            image_path = image_path[:-4] + '.png'
            try:
                image = cv2.imread(image_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            except:
                print(f"Image not found: {image_path}")
                continue

        # Resize the image to a fixed size
        image = cv2.resize(image, (image_width, image_height))
        image = image.astype('float32') / 255.0  # Normalize pixel values between 0 and 1

        images.append(image)
        y.append(label)
    else:
        print(f"File not found: {image_path}")

# Convert the lists to numpy arrays
X = np.array(images)
y = np.array(y)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


datagen = ImageDataGenerator(
    rotation_range=20,  # Randomly rotate the images by up to 20 degrees
    width_shift_range=0.1,  # Randomly shift the images horizontally by up to 10% of the width
    height_shift_range=0.1,  # Randomly shift the images vertically by up to 10% of the height
    zoom_range=0.2,  # Randomly zoom the images by up to 20%
    horizontal_flip=True  # Randomly flip the images horizontally
)

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))

# Freeze the weights of the pretrained layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the pretrained base model
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=20,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping])

# Retrieve the best epoch
best_epoch = early_stopping.stopped_epoch + 1

# Evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
# Load the test set
testset_path = '/2023sumdpl302m/test.csv'
testset_images_folder = '/2023sumdpl302m/testset_images/testset_images'

# Read the CSV file
testset_data = pd.read_csv(testset_path)

# Get the image IDs
test_image_ids = testset_data['image_id'].values

# Prepare empty lists for storing test images and predicted labels
test_images = []
predicted_labels = []

# Prepare empty lists for storing skipped image IDs
skipped_images = []

# Process each image
for image_id in test_image_ids:
    # Construct the image path
    image_path = os.path.join(testset_images_folder, str(image_id))

    # Check if the image path has an extension
    if not any(image_path.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.gif']):
        image_path += '.jpg'  # Add the default extension '.jpg'

    # Load the image using OpenCV
    try:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    except:
        # Retry with PNG extension
        image_path = image_path[:-4] + '.png'
        try:
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        except:
            print(f"Image not found: {image_path}")
            skipped_images.append(image_id)  # Add the skipped image ID to the list
            continue

    # Resize the image to a fixed size
    image = cv2.resize(image, (224, 224))
    image = image.astype('float32') / 255.0  # Normalize pixel values between 0 and 1

    # Append the image to the list
    test_images.append(image)

    # Predict label for the image
    prediction = model.predict(np.expand_dims(image, axis=0))
    predicted_label = int(np.round(prediction)[0])
    predicted_labels.append(predicted_label)

# Create a DataFrame for the predicted labels and image IDs
predicted_df = pd.DataFrame({'id': test_image_ids, 'label': predicted_labels})

# Print the predicted DataFrame
print(predicted_df)

# Print the skipped images
print("Skipped Images:")
for image_id in skipped_images:
    print(f'Image ID: {image_id}')

In [ ]:
predicted_df.to_csv('test3.csv', index=False)